In [ ]:
!pip install pytorch-lightning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt

import nltk
import itertools
import numpy as np
import torch.utils.data as utils
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from nltk.corpus import stopwords
from torch.autograd import Variable
import Data_processing_classifier as dpros
import Data_processing as dpros2
import Emotion_classifier as EM
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import scipy
from sklearn.metrics import confusion_matrix, classification_report

## Data preprocessing

In [ ]:

nltk.download('stopwords')
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

## the list of contractions to substitute

contractions = { 
"ain t": "am not",
"aren t": "are not",
"can t": "cannot",
"can t ve": "cannot have",
" cause": "because",
"could ve": "could have",
"couldn t": "could not",
"couldn t ve": "could not have",
"didn t": "did not",
"doesn t": "does not",
"don t": "do not",
"hadn t": "had not",
"hadn t ve": "had not have",
"hasn t": "has not",
"haven t": "have not",
"he d": "he would",
"he d ve": "he would have",
"he ll": "he will",
"he ll ve": "he will have",
"he s": "he is",
"how d": "how did",
"how d y": "how do you",
"how ll": "how will",
"how s": "how is",
"I d": "I would",
"I d ve": "I would have",
"I ll": "I will",
"I ll ve": "I will have",
"I m": "I am",
"I ve": "I have",
"isn t": "is not",
"it d": "it would",
"it d ve": "it would have",
"it ll": "it will",
"it ll ve": "it will have",
"it s": "it is",
"let s": "let us",
"ma am": "madam",
"mayn t": "may not",
"might ve": "might have",
"mightn t": "might not",
"mightn t ve": "might not have",
"must ve": "must have",
"mustn t": "must not",
"mustn t ve": "must not have",
"needn t": "need not",
"needn t ve": "need not have",
"o clock": "of the clock",
"oughtn t": "ought not",
"oughtn t ve": "ought not have",
"shan t": "shall not",
"sha n t": "shall not",
"shan t ve": "shall not have",
"she d": "she would",
"she d ve": "she would have",
"she ll": "she will",
"she ll ve": "she will have",
"she s": "she is",
"should ve": "should have",
"shouldn t": "should not",
"shouldn t ve": "should not have",
"so ve": "so have",
"so s": "so as",
"that d": "that would",
"that d ve": "that would have",
"that s": "that is",
"there d": "there would",
"there d ve": "there would have",
"there s": "there is",
"they d": "they would",
"they d ve": "they would have",
"they ll": "they will",
"they ll ve": "they will have",
"they re": "they are",
"they ve": "they have",
"to ve": "to have",
"wasn t": "was not",
"we d": "we would",
"we d ve": "we would have",
"we ll": "we will",
"we ll ve": "we will have",
"we re": "we are",
"we ve": "we have",
"weren t": "were not",
"what ll": "what will",
"what ll ve": "what will have",
"what re": "what are",
"what s": "what is",
"what ve": "what have",
"when s": "when is",
"when ve": "when have",
"where d": "where did",
"where s": "where is",
"where ve": "where have",
"who ll": "who will",
"who ll ve": "who will have",
"who s": "who is",
"who ve": "who have",
"why s": "why is",
"why ve": "why have",
"will ve": "will have",
"won t": "will not",
"won t ve": "will not have",
"would ve": "would have",
"wouldn t": "would not",
"wouldn t ve": "would not have",
"y all": "you all",
"y all d": "you all would",
"y all d ve": "you all would have",
"y all re": "you all are",
"y all ve": "you all have",
"you d": "you would",
"you d ve": "you would have",
"you ll": "you will",
"you ll ve": "you will have",
"you re": "you are",
"you ve": "you have"
}


In [ ]:

FILENAME = "CBET.csv"

lines = dpros.read_lines(FILENAME)

train_lines = []
emotions = []


## separate the sentences and the emotions
for (i, line) in enumerate(lines[1:]):
  line_split = dpros.split_line(line)[1:]
  train_lines.append(line_split[0])
  emotion_list = list(map(float, line_split[1:]))
  emotion = emotion_list.index(1)
  emotions.append(emotion)

lines = [dpros.filter_line(line, stopwords_dict, contractions) for line in train_lines]

lines_filtered, list_emotion = dpros.filter_data(lines, emotions)

lines_tokenized = [wordlist.split(' ') for wordlist in lines_filtered]

idx2w, w2idx, freq_dist = dpros.index_(lines_tokenized, vocab_size=dpros.VOCAB_SIZE)

list_lines = dpros.zero_pad(lines_tokenized, w2idx)

metadata = {'w2idx': w2idx, 'idx2w': idx2w, 'limit': dpros.limit}


In [ ]:

## divide the emotions by groups, to get a balance distribution

list_lines_0 = []
list_emotions_0 = []
list_lines_1 = []
list_emotions_1 = []
list_lines_2 = []
list_emotions_2 = []
list_lines_3 = []
list_emotions_3 = []
list_lines_4 = []
list_emotions_4 = []
list_lines_5 = []
list_emotions_5 = []
list_lines_6 = []
list_emotions_6 = []
list_lines_7 = []
list_emotions_7 = []
list_lines_8 = []
list_emotions_8 = []

for i in range(0, 76859):

    if list_emotion[i] == 0:
        list_lines_0.append(list_lines[i])
        list_emotions_0.append(list_emotion[i])
    elif list_emotion[i] == 1:
        list_lines_1.append(list_lines[i])
        list_emotions_1.append(list_emotion[i])
    elif list_emotion[i] == 2:
        list_lines_2.append(list_lines[i])
        list_emotions_2.append(list_emotion[i])
    elif list_emotion[i] == 3:
        list_lines_3.append(list_lines[i])
        list_emotions_3.append(list_emotion[i])
    elif list_emotion[i] == 4:
        list_lines_4.append(list_lines[i])
        list_emotions_4.append(list_emotion[i])
    elif list_emotion[i] == 5:
        list_lines_5.append(list_lines[i])
        list_emotions_5.append(list_emotion[i])
    elif list_emotion[i] == 6:
        list_lines_6.append(list_lines[i])
        list_emotions_6.append(list_emotion[i])
    elif list_emotion[i] == 7:
        list_lines_7.append(list_lines[i])
        list_emotions_7.append(list_emotion[i])
    elif list_emotion[i] == 8:
        list_lines_8.append(list_lines[i])
        list_emotions_8.append(list_emotion[i])



In [ ]:

## divide the sentences and emotions in train, test and validation sets

test_index = int(0.05 * len(list_lines_0))
train_index = int(len(list_lines_0) - test_index)
valid_index = int(train_index - int(0.95 * train_index))
train_index = int(0.95 * train_index)


In [ ]:

lines_train = []
emotion_train = []

lines_valid = []
emotion_valid = []

lines_test = []
emotion_test = []

for i in range(0, train_index):

    lines_train.append(list_lines_0[i])
    lines_train.append(list_lines_1[i])
    lines_train.append(list_lines_2[i])
    lines_train.append(list_lines_3[i])
    lines_train.append(list_lines_4[i])
    lines_train.append(list_lines_5[i])
    lines_train.append(list_lines_6[i])
    lines_train.append(list_lines_7[i])
    lines_train.append(list_lines_8[i])

    emotion_train.append(list_emotions_0[i])
    emotion_train.append(list_emotions_1[i])
    emotion_train.append(list_emotions_2[i])
    emotion_train.append(list_emotions_3[i])
    emotion_train.append(list_emotions_4[i])
    emotion_train.append(list_emotions_5[i])
    emotion_train.append(list_emotions_6[i])
    emotion_train.append(list_emotions_7[i])
    emotion_train.append(list_emotions_8[i])

for j in range(train_index + 1, train_index + valid_index):

    lines_valid.append(list_lines_0[j])
    lines_valid.append(list_lines_1[j])
    lines_valid.append(list_lines_2[j])
    lines_valid.append(list_lines_3[j])
    lines_valid.append(list_lines_4[j])
    lines_valid.append(list_lines_5[j])
    lines_valid.append(list_lines_6[j])
    lines_valid.append(list_lines_7[j])
    lines_valid.append(list_lines_8[j])

    emotion_valid.append(list_emotions_0[j])
    emotion_valid.append(list_emotions_1[j])
    emotion_valid.append(list_emotions_2[j])
    emotion_valid.append(list_emotions_3[j])
    emotion_valid.append(list_emotions_4[j])
    emotion_valid.append(list_emotions_5[j])
    emotion_valid.append(list_emotions_6[j])
    emotion_valid.append(list_emotions_7[j])
    emotion_valid.append(list_emotions_8[j])

for k in range(train_index + valid_index + 1, len(list_lines_0) - 1):

    lines_test.append(list_lines_0[k])
    lines_test.append(list_lines_1[k])
    lines_test.append(list_lines_2[k])
    lines_test.append(list_lines_3[k])
    lines_test.append(list_lines_4[k])
    lines_test.append(list_lines_5[k])
    lines_test.append(list_lines_6[k])
    lines_test.append(list_lines_7[k])
    lines_test.append(list_lines_8[k])

    emotion_test.append(list_emotions_0[k])
    emotion_test.append(list_emotions_1[k])
    emotion_test.append(list_emotions_2[k])
    emotion_test.append(list_emotions_3[k])
    emotion_test.append(list_emotions_4[k])
    emotion_test.append(list_emotions_5[k])
    emotion_test.append(list_emotions_6[k])
    emotion_test.append(list_emotions_7[k])
    emotion_test.append(list_emotions_8[k])

train_dataset = utils.TensorDataset(torch.LongTensor(lines_train), torch.LongTensor(emotion_train))
valid_dataset = utils.TensorDataset(torch.LongTensor(lines_valid), torch.LongTensor(emotion_valid))
test_dataset = utils.TensorDataset(torch.LongTensor(lines_test), torch.LongTensor(emotion_test))


## Emotion classifier

In [ ]:

class LSTMClassifier(pl.LightningModule):

    def __init__(self, vocab_size, embedding_length, output_dim, hid_dim, dropout):
        super(LSTMClassifier, self).__init__()

        self.attention = EM.SelfAttention(hid_dim, dropout)
        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.lstm = nn.LSTM(embedding_length, hid_dim, bidirectional=True)
        self.hid_dim = hid_dim

        self.drop_1 = nn.Dropout(dropout)
        self.drop_2 = nn.Dropout(dropout)
        self.pred = nn.Linear(hid_dim * 2, output_dim) ## output_dim -> 9

    def forward(self, inp):

        embedded_inp = self.drop_1(self.word_embeddings(inp)) ## embedded_inp.size() = (batch size, sequence length, embedding size)

        embedded_inp = embedded_inp.permute(1, 0, 2) ## embedded_inp.size() = (sequence length, batch size, embedding size)
        
        h_0 = Variable(torch.zeros(2, embedded_inp.size(1), self.hid_dim)).cuda() ## h_0.size() = (num directions, batch size, hidden size)
        c_0 = Variable(torch.zeros(2, embedded_inp.size(1), self.hid_dim)).cuda() ## c_0.size() = (num directions, batch size, hidden size)

        output, (final_hidden_state, final_cell_state) = self.lstm(embedded_inp, (h_0, c_0))

        attention_product = self.attention(output, final_hidden_state)

        pred = self.drop_2(self.pred(attention_product)) ## pred.size() = (batch size, output size)

        return pred

    def training_step(self, batch, batch_idx):

        src, emotion = batch

        output = self.forward(src)

        loss = criterion(output.contiguous(), emotion)

        return {'loss': loss}

    def training_epoch_end(self, outputs):

        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print("Train loss: ", avg_loss)
        return {'train_loss': avg_loss}

    def validation_step(self, batch, batch_idx):

        src, emotion = batch

        output = self.forward(src)

        val_loss = criterion(output.contiguous(), emotion)

        return {'val_loss': val_loss}

    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Avg loss: ", avg_loss)
        return {'val_loss': avg_loss}

    def test_step(self, batch, batch_idx):

        src, emotion = batch

        output = self.forward(src)

        test_loss = criterion(output.contiguous(), emotion)

        return {'test_loss': test_loss}

    def test_epoch_end(self, outputs):

        test_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss}

    def configure_optimizers(self):

        return optim.Adam(model.parameters(), lr=0.0005)

    def train_dataloader(self):

        train_loader = DataLoader(train_dataset, batch_size=dpros2.batch_size, num_workers=3, drop_last=True, shuffle=True)

        return train_loader

    def val_dataloader(self):

        valid_loader = DataLoader(valid_dataset, batch_size=dpros2.batch_size, num_workers=3, drop_last=True)

        return valid_loader

    def test_dataloader(self):

        test_loader = DataLoader(test_dataset, batch_size=dpros2.batch_size, num_workers=3, drop_last=True)

        return test_loader


In [ ]:

INPUT_DIM = len(metadata['idx2w'])
OUTPUT_DIM = 9
EMB_DIM = 200
HID_DIM = 256
DROPOUT = 0.45
SEQ_LEN = dpros.limit['maxs']


In [ ]:

criterion = nn.CrossEntropyLoss().cuda()


In [ ]:

## names of the emotions, for the confusion matrix

names = ('anger','fear','joy','love','sadness','surprise','thankfulness','disgust','guilt')


In [ ]:

model = LSTMClassifier(INPUT_DIM, EMB_DIM, OUTPUT_DIM, HID_DIM, DROPOUT)

checkpoint_callback = ModelCheckpoint(filepath='drive/My Drive/Colab Notebooks/Classifier/', save_top_k=1, verbose=True, monitor='val_loss',mode='min')

trainer = pl.Trainer(gpus=1, max_epochs=100, log_save_interval=100000, weights_summary=None, progress_bar_refresh_rate=0, log_gpu_memory=None, benchmark=True, default_root_dir='drive/My Drive/Colab Notebooks/Classifier/', checkpoint_callback=checkpoint_callback)

## trainer = pl.Trainer(resume_from_checkpoint='_ckpt_epoch_27.ckpt', gpus=1, max_epochs=28, log_save_interval=100000, weights_summary=None, progress_bar_refresh_rate=0, log_gpu_memory=None, benchmark=True, default_root_dir='drive/My Drive/Colab Notebooks/Classifier/', checkpoint_callback=checkpoint_callback)


In [ ]:

trainer.fit(model)
trainer.test(model)


In [ ]:

## this function outputs the emotion predictions of the model for different sentences

def predict(model, src):

    class_output = model(src)
    output_softmax = F.softmax(class_output, -1)
    predicted = torch.max(output_softmax.data, -1)
    predicted = predicted.indices

    return predicted


In [ ]:

## this is for making the predictions for the test set. Then, we store the predictions and target emotions in two lists

test_loader = DataLoader(test_dataset, batch_size=dpros.BATCH_SIZE, drop_last=True)

target = list()
predicted = list()

model.freeze()

for i, data in enumerate(test_loader, 0):

    src, emotion = data

    predicted_emotion = predict(model.cuda(), src.cuda())

    for elem in predicted_emotion:

      predicted.append(elem.cpu())

    for elem2 in emotion:

      target.append(elem2)


In [ ]:

## plot the confusion matrix for the test set

print(classification_report(predicted, target, digits=10))

cm = confusion_matrix(target, predicted)

plt.figure(figsize=(10,10))
EM.plot_confusion_matrix(cm, names)
